In [41]:
from urllib.request import urlopen
from urllib.parse import urljoin
from urllib.parse import urlparse, parse_qs
import requests
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import time

In [68]:
position = ['QB', 'RB', 'WR', 'TE', 'DST', 'K']
week = ['1','2','3','4', '5','6','7','8','9','10','11','12','13','14','15','16','17']
season = ['2012', '2013','2014','2015','2016']


In [21]:
columns = ['player_id', 'name', 'team', 'vs', 'ffpts', 'patt', 'pcmp', 'pyds', 'ptds', 'pint', 'p2', 'ruatt', 'ruyds','rutds', 'ru2', 'rec', 'recyds', 'rectd', 'rec2', 'fum', 'fmbtd']
pos_week = pd.DataFrame(columns)

In [39]:
for val in position:
    urlparse('http://www.footballdb.com/fantasy-football/index.html?pos={}&yr=2012&wk=1&rules=1')
    query = 'pos={}&yr=2012&wk=1&rules=1'.format(val)
    r = requests.get(urljoin('http://www.footballdb.com/fantasy-football/index.html?pos={}&yr=2012&wk=1&rules=1', query))
    html = r.content
    bsobj = BeautifulSoup(html, 'lxml')
    print(bsobj)
#     val= []
#     for item in (bsobj.tbody({'a':'href'})):
#         if '.' not in item.get_text():
#             val.append(item.get_text())

#     qb_1['name'] = qb_list
#     qb_1['player_id'] = qb.index + 1


<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL /fantasy-football/pos=QB&amp;yr=2012&amp;wk=1&amp;rules=1 was not found on this server.</p>
<p>Additionally, a 404 Not Found
error was encountered while trying to use an ErrorDocument to handle the request.</p>
<hr/>
<address>Apache/2.2.31 (Amazon) Server at www.footballdb.com Port 80</address>
</body></html>

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL /fantasy-football/pos=RB&amp;yr=2012&amp;wk=1&amp;rules=1 was not found on this server.</p>
<p>Additionally, a 404 Not Found
error was encountered while trying to use an ErrorDocument to handle the request.</p>
<hr/>
<address>Apache/2.2.31 (Amazon) Server at www.footballdb.com Port 80</address>
</body></html>

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Fou

In [78]:
qb_list = []
rb_list = []
wr_list = []
te_list = []
k_list = []
dst_list = []
for pos in position:
    url = ('http://www.footballdb.com/fantasy-football/index.html?pos=qb&yr=2012&wk=1&rules=1').format(pos)
    html = urlopen(url)
    bsobj= BeautifulSoup(html, 'lxml')
    for item in (bsobj.tbody({'a':'href'})):
        (if pos == 'QB') & (if '.' not in item.get_text()):
            qb_list.append(item.get_text())
        elif pos == 'RB' & if '.' not in item.get_text():
            rb_list.append(item.get_text())
        elif pos == 'TE' & if '.' not in item.get_text():
            te_list.append(item_get_text())
        elif pos == 'WR' & if '.' not in item.get_text():
            wr_list.append(item.get_text())
        elif pos = 'K' & if '.' not in item.get_text():
            k_lsit.append(item.getText())
        elif pos == 'DST' & if  '.' not in item.get_text():
            dst_list.append(item.get_text())
            

SyntaxError: invalid syntax (<ipython-input-78-550131c043ad>, line 12)

In [60]:
html = urlopen('http://www.footballdb.com/fantasy-football/index.html?pos=qb&yr=2012&wk=1&rules=1')
bsobj= BeautifulSoup(html, 'lxml')
print(bsobj.tbody({'a':'href'}))

[<a href="/players/matt-ryan-ryanma01">Matt Ryan</a>, <a href="/players/matt-ryan-ryanma01">M. Ryan</a>, <a href="/players/tony-romo-romoto01">Tony Romo</a>, <a href="/players/tony-romo-romoto01">T. Romo</a>, <a href="/players/robert-griffin-griffro02">Robert Griffin III</a>, <a href="/players/robert-griffin-griffro02">R. Griffin</a>, <a href="/players/drew-brees-breesdr01">Drew Brees</a>, <a href="/players/drew-brees-breesdr01">D. Brees</a>, <a href="/players/mark-sanchez-sanchma01">Mark Sanchez</a>, <a href="/players/mark-sanchez-sanchma01">M. Sanchez</a>, <a href="/players/aaron-rodgers-rodgeaa01">Aaron Rodgers</a>, <a href="/players/aaron-rodgers-rodgeaa01">A. Rodgers</a>, <a href="/players/peyton-manning-mannipe01">Peyton Manning</a>, <a href="/players/peyton-manning-mannipe01">P. Manning</a>, <a href="/players/blaine-gabbert-gabbebl01">Blaine Gabbert</a>, <a href="/players/blaine-gabbert-gabbebl01">B. Gabbert</a>, <a href="/players/joe-flacco-flaccjo01">Joe Flacco</a>, <a href="/

In [29]:
for val in pos:
    r = requests.get('http://www.footballdb.com/fantasy-football/index.html?pos={}&yr=2012&wk=1&rules=1').format(val)


AttributeError: 'Response' object has no attribute 'format'

In [10]:
qb_1

NameError: name 'qb_1' is not defined

In [9]:
columns = ['player_id', 'name', 'team', 'vs', 'ffpts', 'patt', 'pcmp', 'pyds', 'ptds', 'pint', 'p2', 'ruatt', 'ruyds','rutds', 'ru2', 'rec', 'recyds', 'rectd', 'rec2', 'fum', 'fmbtd']

#Load page and establish beautiful soup object
for val in wk:
    html= urlopen('http://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2012&wk=wk&rules=1')
    bsobj_wk= BeautifulSoup(html, 'lxml')
    pos_wk = create_df(qb,wk)
    

NameError: name 'qb' is not defined

NameError: name 'qb_10' is not defined

In [4]:
qb_1

,player_id,name,team,vs,ffpts,patt,pcmp,pyds,ptds,pint,...,ruatt,ruyds,rutds,ru2,rec,recyds,rectd,rec2,fum,fmbtd


In [88]:
#Parse html to obtain list of QB's competeing in rleevant week 
qb_list= []
for item in (bsobj.tbody({'a':'href'})):
    if '.' not in item.get_text():
        qb_list.append(item.get_text())

qb_1['name'] = qb_list
qb_1['player_id'] = qb.index + 1

In [89]:
#Parse beautiful soup object to get only data needed in list format
parsed_stats = []
for item in bsobj.tbody.find_all({'td':'class'}):
    parsed_stats.append(item.get_text().split())

In [91]:
#Populate team and vs columns in qb dataframe by sorting parsed_list data
stats = []
team_list = []
vs_list = []
for idx in range(len(parsed_stats)):
    try:
        stats.append(float(parsed_stats[idx][0]))
    except:
        if len(parsed_stats[idx]) > 1:
            team_list.append(parsed_stats[idx][4])
            vs_list.append(parsed_stats[idx+1][0].replace('@', ''))

qb_1.team, qb_1.vs = team_list, vs_list

remaining_columns = columns[4:]
for num, col in enumerate(remaining_columns, start=2):
    print(num, col)
    # Flatten the list and assign it to the relevant column
    flat_list = [item for sublist in parsed_stats[num:][::19] for item in sublist]
    qb_1[col] = flat_list

2 ffpts
3 patt
4 pcmp
5 pyds
6 ptds
7 pint
8 p2
9 ruatt
10 ruyds
11 rutds
12 ru2
13 rec
14 recyds
15 rectd
16 rec2
17 fum
18 fmbtd


In [92]:
qb_1

,player_id,name,team,vs,ffpts,patt,pcmp,pyds,ptds,pint,...,ruatt,ruyds,rutds,ru2,rec,recyds,rectd,rec2,fum,fmbtd
0,1,Matt Ryan,Atl,KC,37.00,31,23,299,3,0,...,3,25,1,0,0,0,0,0,0,0
1,2,Tony Romo,Dal,NYG,29.00,29,22,307,3,1,...,5,12,0,0,0,0,0,0,0,0
2,3,Robert Griffin III,"Griffin,",NO,28.00,26,19,320,2,0,...,9,42,0,0,0,0,0,0,0,0
3,4,Drew Brees,NO,Was,27.00,52,24,339,3,2,...,0,0,0,0,0,0,0,0,0,0
4,5,Mark Sanchez,NYJ,Buf,26.00,27,19,266,3,1,...,0,0,0,0,0,0,0,0,0,0
5,6,Aaron Rodgers,GB,SF,26.00,44,30,303,2,1,...,5,27,0,0,0,0,0,0,0,0
6,7,Peyton Manning,Den,Pit,24.00,26,19,253,2,0,...,4,3,0,0,0,0,0,0,0,0
7,8,Blaine Gabbert,Jax,Min,23.00,39,23,260,2,0,...,5,6,0,0,0,0,0,0,1,0
8,9,Joe Flacco,Bal,Cin,23.00,29,21,299,2,0,...,2,-1,0,0,0,0,0,0,0,0
9,10,Jay Cutler,Chi,Ind,23.00,35,21,333,2,1,...,4,-4,0,0,0,0,0,0,0,0
